# Random walk のアルゴリズムとデータ構造
最適化に際し，アルゴリズムとデータ構造の選択は重要である． $O(n^2)$ をどれだけ速くしたところで $O(n)$ を上回ることはなく，効率の悪いデータ構造はアルゴリズムの性能を低下させる恐れがある．

## アルゴリズム

二部グラフ上の Random walk を利用したレコメンドを考える．
あるノードを始点とする1ステップは，

1. 近傍の取得
2. 乱数による近傍の選択
3. ノード遷移

から構成される．これを一定回数繰り返し，到達したノードをカウントする．集計して得られた値をクエリに対する各ノードの関連度とする．
ここで，現在地点は一定確率 $\alpha$ でリセットされる．

```python
curr_node = query_node
for i in range(n_steps):
    curr_node = curr_node.get_random_neighbor()
    curr_node = curr_node.get_random_neighbor()
    curr_node.visit_count += 1
    if random() < alpha:
        curr_node = query_node
```

## データ構造
あるノードの近傍の取得は，2種類の array によって実現できる．

- **adjacency**:
ノードの近傍を保持したリストをノードID順に連結した array

- **offsets**:
ノードの次数をノードID順に累積した array

ノード n の次数と近傍の取得はそれぞれ定数時間で達成できる.

```python
degree = offsets[n+1] - offsets[n]
neighbor = adjacency[offsets[n]:offsets[n+1]]
```



(参考:adjacency と offsets の生成)

df を インタラクションを表すタプル( `(<user_id>, <item_id>)` )のリストとしたとき，
```python
ui_adj = df.groupby("user_id").item_id.apply(list)
iu_adj = df.groupby("item_id").user_id.apply(list)
adjacency = pd.concat([ui_adj, iu_adj])
offsets = adjacency.apply(len)
offsets = np.cumsum([0] + offsets.tolist())
adjacency = np.concatenate(adjacency.tolist())
```

# Python による実装

In [ ]:
!cat randomwalk/rw.py

In [ ]:
%run -t run.py model/ -m vanilla

# プロファイリング
最適化に向けてプロファイリングを行う．ここでは `cProfile` と `line_profiler` を用いる．

## cProfile
すべての関数の実行時間を測定する．

In [ ]:
# 見辛いのでターミナルでの実行を推奨
# python -m cProfile -s time run.py model/ -m vanilla > cprof
%run -p run.py model/ -m vanilla

In [ ]:
!head -n 20 cprof

近傍の取得 `get_neighbors` が 1,437,131 回呼ばれ，合計 1.6 秒費やしていることが確認できる。
近傍のサンプリング `sample_neighbor` の内部で `get_neighbors` が呼ばれるため， `get_neighbors` がボトルネックであることが特定できた．

## line_profiler
特定の関数について，行単位の実行時間を測定する．
既にボトルネックは特定できているが，関数の詳細なプロファイリングによって関数全体に対するボトルネックの実行時間の割合を明らかにすることで改善の余地を見積もることができる．

**ターミナルで実行**

プロファイルを取得する関数に `@profile` デコレータを付与する．

```python
    @profile
    def pixie_random_walk(self, q, steps):
        visit_count = Counter()
        total_steps = 0
        n_high_visited = 0

        while (...
```

その後，ターミナルから以下コマンドを実行する．
            
```bash
kernprof -l -v run.py model/ -m vanilla > lprof
```

**スクリプトから実行**

デコレータは不要．以下を実行する．

In [ ]:
import line_profiler
from run import main
from randomwalk.rw import RandomWalk

profile = line_profiler.LineProfiler(RandomWalk().pixie_random_walk)
profile.runcall(main, "model/", "vanilla")
profile.print_stats()

`pixie_random_walk` 関数の while 文内部で `sample_neighbor` が2回呼び出され，合計で 50% 強の時間を費やしていることが確認できた．

## ボトルネックの改善効果を見積もる

ボトルネックの関数全体に対する寄与を定量化した．この値とアムダールの法則により関数全体の実行時間改善率を見積もることができる．
$S_T$ を関数全体の改善率， $P$ を最適化対象部分の全実行時間に対する割合， $S_P$ を最適化した部分 $P$ の改善率として，

$$
S_T = \frac{1}{(1-P) + \frac{P}{S_P}}
$$

.仮に `sample_neighbor` を 50% 改善したとすると，関数全体の改善率は，

$$
\frac{1}{(1-0.5) + \frac{0.5}{1.5}} = 1.2
$$

となる.ボトルネックを 50% 改善できたとしても関数全体では 20% の改善に留まる． `cProfile` の結果を参考値として用いると( `cProfile` のほうが `line_profiler` よりオーバーヘッドが小さい)， `pixie_random_walk` の実行時間 5.4 秒が 1 秒程度短縮される可能性がある．

目標の実行速度から必要な改善率を求め，最適化戦略を組み立てる必要がある．

In [ ]:
1/(0.5 + 1./3)

# 結果的に効果が薄かった最適化
プロファイラの指摘を直接改善するものではないが，手軽に着手可能な最適化を試す．

## LRU キャッシュ
Web データから構築したグラフにはハブが存在することが多い．すなわち，特定のノードはアクセス頻度が大きい．このようなデータ特性を持つ場合， キャッシュが効く可能性がある．今回は LRU キャッシュを適用する．

[LRU キャッシュ](https://ja.wikipedia.org/wiki/%E3%82%AD%E3%83%A3%E3%83%83%E3%82%B7%E3%83%A5%E3%82%A2%E3%83%AB%E3%82%B4%E3%83%AA%E3%82%BA%E3%83%A0#.E4.BE.8B)
> Least Recently Used (LRU): 最近最も使われていないデータを最初に捨てる。このアルゴリズムでは、どのデータがどの時点で使われたかという情報を保持する必要があり、本当に最近最も使われていないデータを常に捨てるようにしようとすると、かなり手間がかかる。一般的実装では、キャッシュライン毎に世代ビット群（age bits）を持たせ、どのラインが最近最も使われていないかを世代ビット群で判断する。その場合、あるキャッシュラインを使うときには、常に全キャッシュラインの世代ビット群を更新する必要がある。

[@functools.lru_cache](https://docs.python.org/ja/3/library/functools.html#functools.lru_cache)
> LRU (least recently used) キャッシュ は、最新の呼び出しが次も呼び出される可能性が最も高い場合 (例えば、ニュースサーバーの最も人気のある記事は、毎日変わる傾向にあります) に最も効率が良くなります。キャッシュのサイズ制限は、キャッシュがウェブサーバーなどの長期間に渡るプロセスにおける限界を超えては大きくならないことを保証します。

> 一般的には、 LRU キャッシュは前回計算した値を再利用したいときにのみ使うべきです。 そのため、副作用のある関数、呼び出すごとに個別の可変なオブジェクトを作成する必要がある関数、 time() や random() のような純粋でない関数をキャッシュする意味はありません。

Python の LRU キャッシュ は `functools` に実装されている．キャッシュ対象の関数をデコレートして有効化する．

```python
@lru_cache(maxsize=None)
def get_neighbors(self, node):
    min_id = self._offsets[node]
    max_id = self._offsets[node + 1]
    return self._adjacency[min_id:max_id]
```

In [ ]:
# 編集箇所は 3 行のみ
!diff randomwalk/rw.py randomwalk/rw_cache.py

In [ ]:
%run -t run.py model/ -m cache

`run_random_walk` の平均実行時間に若干の改善が見られた．

## 並列処理
一応並列処理も試みる．

`run_random_walk` では，クエリノードの近傍全てについて `pixie_random_walk` を実行している．ここで，各実行を  `multiprocessing` モジュールで並列化する．

```python
with Pool() as pool:
    results = pool.starmap_async(
        self.pixie_random_walk, zip(query_items, steps_per_query)
    ).get()
```

In [ ]:
%run -t run.py model/ -m parallel

`run_random_walk` の平均実行時間は悪化する結果となった．ここで，今回の並列化箇所はアルゴリズムの性質上あまり有効でないと思われる．

並列実行の最適化次第では改善の可能性は存在するものの，プロファイラが指摘した改善とは異なるのでこれ以上は扱わない．

# Cython
プロファイリングにより，ボトルネックは `numpy.ndarray` の要素へのアクセスであることがわかっている．Pythonレベルでこれ以上の改善は期待できないため， Cython の導入を検討する．

## Cython とは
- Python に C の静的型付けシステムを導入するプログラミング言語
- Cython によって記述されたソースコードを C または C++ のソースコードに変換するコンパイラ

Python は動的インタプリタ言語であり，直感的で柔軟である．柔軟性と安全性を両立するために Python のAPIの背後にはバリデーション処理などが厚く存在し，これがオーバーヘッドとなっている．このため，静的型付け言語と比較すると低速となる．

Cython は Python によく似た文法で記述することが可能であり， Cython で記述されたソースコードは `cython` コンパイラが C コードに変換する．変換後は Python の余計な安全装置などが取り除かれ，必要最低限な API だけが残るため，純粋な C や C++ に近い性能を発揮する．

## Cython 実装ファイルとコンパイル
### 実装ファイル
Cython の実装ファイルには `.pyx` という拡張子が付く．これの他に，宣言ファイルとして `.pxd` という拡張子が存在する． C++ のヘッダーファイルに近い機能を提供するが，今回は扱わない．

まずは呼び出しの多い `sample_neighbor` 関数を対象として Cython 化を試みる． `c_sample_neighbor.pyx` というファイルを作成して以下の通り記述する．

In [ ]:
!cat randomwalk/c_sample_neighbor.pyx

Python から呼ばれる関数は `cpdef` で定義する．通常， Cython の関数定義は `cdef` キーワードを用いるが， `cdef` で定義された関数には Cython からしかアクセスできない．今回定義した `_c_sample_neighbor` は 外部の Python コードから呼ぶことを想定しているため， `cpdef` で宣言して Python からの呼び出しを許可する．

`long[:] adjacency` という構文は `adjacency` が型付き memoryview 型であることを宣言する． memoryview 型はバッファの操作を目的とする Python の組み込み型であり， `numpy.ndarray` もこれをサポートする．重要なのは， memoryview 型で受ければ `numpy.ndarray` をデータのコピーなしに Cython に渡せるということである． `adjacency` は非常に巨大な array であるが，受け渡しのオーバーヘッドは小さい．

関数内部で使用するローカルな変数は `cdef` ブロックで宣言する．ここですべての変数を宣言することが重要である．宣言がなくてもプログラムは問題なく実行されるが，宣言されていない変数に対しては背後で Python の型推論が実行されるので Cython の恩恵を受けられない．

`_c_sample_neighbor` を呼び出す Python ファイルは大きな編集を必要としない． Cython モジュールをまるで Python モジュールであるかのように利用できる．

```python
# モジュールのインポート
from c_sample_neighbor import _c_sample_neighbor
```
```python
# モジュールの呼び出し
curr_user = _c_sample_neighbor(
    self._adjacency, self._offsets, curr_item
)
```

### コンパイル
コンパイルは `setup.py` によって行う．以下の通り記述された `setup.py` を配置する．

```python
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules=cythonize("randomwalk/c_sample_neighbor.pyx")) 
```

コンパイルの実行は以下のコマンドで行う．

```bash
python setup.py build_ext --inplace
```

引数 `build_ext` は `Extension` オブジェクトを拡張モジュールにビルドする指示を出す. `--inplace` オプションは拡張モジュールを実装ファイルと同じディレクトリに配置する指示を出す．

コンパイルを実行すると `randomwalk/` に `c_sample_neighbor.c` というファイルが生成されている．開いてみると，何やら壮絶な定義文が綴られているが C のソースコードであることはわかる．

In [ ]:
!ls -ltr randomwalk/ | tail -n 1 

In [ ]:
!cat randomwalk/c_sample_neighbor.c

これまでと同様の手順で実行して時間を計測する．

In [ ]:
%run -p run.py model/ -m cy

## 再度の line_profiler
`sample_neighbor` を Cython 化したことによって `pixie_random_walk` の挙動がどの様に変化したかを調べるために `line_profiler` を実行する．

In [ ]:
from randomwalk.rw_cy import RandomWalk

profile = line_profiler.LineProfiler(RandomWalk().pixie_random_walk)
profile.runcall(main, "model/", "cy")
profile.print_stats()

`pixie_random_walk` 実行時間全体に占める `sample_neighbor` の割合が 40% 弱に減少したことが確認できる( try-except 構文から if 文に変更した分も影響する)．確かに改善はしたが，効果は大きいとは言えない．さらなる改善に向けて新しいツールを導入する．

## Python APIの発見(コードアノテーション)
Cython は Python と C (や C++) のハイブリッド言語であり， C の API が扱えない部分に関しては Python の APIが実行される．
例えば先に述べたとおり，宣言がされていない変数に対しては Python の型推論が実行される．
この通り， C-Python 間の解釈が自動化されているため，意図せず Python API を呼び出している場合が存在し，性能低下の原因となる．

これを解決するために，関数中の Python API を探すためのツールとして Cython はコードアノテータと呼ばれる機能を提供する．

重要なのは，プロファイラは関数の「どこが遅いのか」を特定するツールであり，コードアノテータは「なぜ遅いのか」を特定するツールであるということである．

コードアノテータは以下のコマンドで実行する．

```bash
cython --annotate randomwalk/c_sample_neighbor.pyx
```

これを実行すると， `randomwalk` 配下に `c_sample_neighbor.html` というファイルが生成される．

In [ ]:
!open randomwalk/c_sample_neighbor.html

注目すべきは 3 行目と 17 行目である． `_c_sample_neighbor` は Python からの呼び出しを想定しているため，関数の入り口と出口で必ず C と Python の変換が実行される．行を展開すると変換処理の厚みを確認できる．そして `_c_sample_neighbor` はループの中で実行されるため，この処理はイテレーションのたびに実行される．コードアノテーションによって Cython の効果を最大化できていないことがわかった．

ちなみに 11,12 行目の薄いラインは `IndexError` のバリデーションに起因する． `IndexError` が発生しないことがわかっている場合，以下のように関数をデコレートしてバリデーションを排除することができる．

```python
@cython.boundscheck(False)
@cython.wraparound(False)
cdef long _c_sample_neighbor(
    ...
```

## 関数全体の Cythonize
変換処理が低速化の原因であることから，高速化を望むのであればループをすべて Cython で記述するしかない． `c_randomwalk.pyx` では Random walk のほぼすべての処理を Cython で記述している．コードアノテータを利用して極力 Python API を排除した．

```bash
cython --annotate randomwalk/c_randomwalk.pyx
```

これを実行すると実行時間が劇的に改善されていることが確認できる．

In [ ]:
%run -p run.py model/ -m cython

# 参考
- [ハイパフォーマンスPython](https://www.amazon.co.jp/dp/4873117402)
- [Cython ―Cとの融合によるPythonの高速化](https://www.amazon.co.jp/dp/4873117275)
- [Optimized C++ ―最適化、高速化のためのプログラミングテクニック](https://www.amazon.co.jp/dp/4873117925)
- [scikit-learn](https://scikit-learn.org/stable/)
- [Eksombatchai et al. 2018](https://labs.pinterest.com/user/themes/pin_labs/assets/paper/paper-pixie.pdf)
- [Paudel et al. 2016](https://dl.acm.org/doi/10.1145/2955101)